In [ ]:
from __future__ import print_function
from beakerx import *
import pandas as pd
import sys
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
import dalmatian as dm
import numpy as np

sys.path.insert(0, '../')
from src.CCLE_postp_function import *
sys.path.insert(0, '../../JKBio/')
import Datanalytics as da
import TerraFunction as terra

from taigapy import TaigaClient
from IPython.display import Image, display
tc = TaigaClient()

In [ ]:
refnamespace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"

# Downloading from terra

In [ ]:
refwm= dm.WorkspaceManager(refnamespace)

Let's get samples from when we made the switch to CGA only data

In [ ]:
res = refwm.get_sample_sets().loc['DM19Q2_COMPLETE_SET']
res

In [ ]:
filtered = res['passedCGA_filteredMAF_aggregated']
unfiltered = res['unfiltered_CGA_MAF_aggregated']

! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"
! gsutil cp $unfiltered "temp/mutation_unfiltered_terra_merged.txt"

let's also look at the latest data version to see if the problem comes from here

In [ ]:
release = pd.read_csv('../temp/mutations.19Q3.tsv', sep='\t')

In [ ]:
len(release)

In [ ]:
filtered = pd.read_csv('../temp/mutation_filtered_terra_merged.txt', sep='\t')
unfiltered = pd.read_csv('../temp/mutation_unfiltered_terra_merged.txt', sep='\t')

In [ ]:
len(filtered)

let's only use the same set of columns for comparison purposes

In [ ]:
unfiltered = unfiltered[set(unfiltered.columns.tolist()) & set(filtered.columns.tolist())]
filtered = filtered[unfiltered.columns]

## How much genome change non overlap do we have to the filtered version?

{{b}}

In [ ]:
b = len(set(filtered.Genome_Change.tolist())) - len(set(unfiltered.Genome_Change.tolist()) & set(filtered.Genome_Change.tolist()))
b

## how much genome change do we have in duplicate? 
R : {{a*100}}%

In [ ]:
a = (len(filtered) - len(set(filtered.Genome_Change.tolist()))) / len(filtered)
a

does not seem to be a 19Q3 problem

In [ ]:
len(filtered[filtered.Hugo_Symbol=='TP53']) - len(release[release.Hugo_Symbol=='TP53'])

In [ ]:
set(filtered[filtered.Hugo_Symbol=='TP53'].Protein_Change) - set(release[release.Hugo_Symbol=='TP53'].Protein_Change)

In [ ]:
filtered.columns

In [ ]:
filtered.iloc[0].values

In [ ]:
release.columns

In [ ]:
filtered[filtered.Protein_Change=='p.PSS126fs'].values

In [ ]:
set(unfiltered[unfiltered.Hugo_Symbol=='TP53'].Protein_Change)

In [ ]:
set(filtered[filtered.Hugo_Symbol=='TP53'].Protein_Change)

In [ ]:
set(release[release.Hugo_Symbol=='TP53'].Protein_Change)

## Looking at data from 19Q1 to see exactly the missign mutations

In [ ]:
new = tc.get(name='depmap-mutation-calls-9be3', version=15, file='depmap_19Q3_mutation_calls')
prev = tc.get(name='depmap-mutation-calls-9be3', version=11, file='depmap_19Q1_mutation_calls')

In [ ]:
prev= prev.drop('Unnamed: 0',1)

In [ ]:
prev.columns

lets only look at the data of cell lines that we had in VA+ CGA vs CGA

In [ ]:
newp = new[new.DepMap_ID.isin(set(prev.DepMap_ID))]

In [ ]:
unfiltered['DepMap_ID'] =[i.split('_')[1] for i in unfiltered['Tumor_Sample_Barcode']]

In [ ]:
unfiltered = unfiltered[unfiltered.DepMap_ID.isin(set(prev.DepMap_ID))]

we still have more data on the unfiltered version (seem to make sense)

In [ ]:
len(unfiltered) - len(prev)

there seem to be a lot less data in the new version compared to the old one 

In [ ]:
len(newp) - len(prev)

In [ ]:
len(newp[newp.Hugo_Symbol=='TP53'])

In [ ]:
len(prev[prev.Hugo_Symbol=='TP53'])

there is cell lines for which there is mutations that have been called in VA +CGA but not in CGA. And here are the genome changes that have never been called in the CGA only version

In [ ]:
set(prev[prev.Hugo_Symbol=='TP53'].DepMap_ID) - set(newp[newp.Hugo_Symbol=='TP53'].DepMap_ID) 

In [ ]:
other = set(prev[prev.Hugo_Symbol=='TP53'].Genome_Change) - set(newp[newp.Hugo_Symbol=='TP53'].Genome_Change) 
other

In [ ]:
cgamva = set(newp[newp.Hugo_Symbol=='TP53'].Genome_Change) - set(prev[prev.Hugo_Symbol=='TP53'].Genome_Change) 
cgamva

In [ ]:
new[new.Hugo_Symbol=='TP53'][new.Genome_Change.isin(cgamva)]

let's look at the tp53 mutations with this particular genome change.

In [ ]:
a = pd.read_csv('../../../../../new.tsv', sep='\t')
b = a[a.Hugo_Symbol=='TP53']

In [ ]:
b.T.to_csv('new.csv')

In [ ]:
prev.columns.tolist()

In [ ]:
prev[prev.Hugo_Symbol=='TP53'][prev.Genome_Change.isin(other)]

In [ ]:
tp53prev = prev[prev.Hugo_Symbol=='TP53']

In [ ]:
newp[newp.Hugo_Symbol=='TP53'][newp.Genome_Change.isin(set(tp53prev.Genome_Change)) & newp.DepMap_ID.isin(set(tp53prev.DepMap_ID))]

In [ ]:
found = []
for i, v in newp[newp.Hugo_Symbol=='TP53'].iterrows():
    print(i,end='\r')
    for k, val in prev[prev.Hugo_Symbol=='TP53'].iterrows():
        if val.Genome_Change == v.Genome_Change:
            if val.DepMap_ID == v.DepMap_ID:
                found.append(i)
                break

In [ ]:
innewnotinprev = newp[~newp.index.isin(found)][newp.Hugo_Symbol=='TP53']

In [ ]:
len(innewnotinprev)

In [ ]:
innewnotinprev.isCOSMIChotspot.sum()

In [ ]:
len([i for i in inprevnotinnew.VA_WES_AC.values if type(i) is str])

In [ ]:
found = []
for k, val in prev[prev.Hugo_Symbol=='TP53'].iterrows():
    print(k,end='\r')
    for i, v in unfiltered[unfiltered.Hugo_Symbol=='TP53'].iterrows():
        if val.Genome_Change == v.Genome_Change:
            if val.DepMap_ID == v.DepMap_ID:
                found.append(k)
                break

In [ ]:
inprevnotinunfiltered = prev[~prev.index.isin(found)][prev.Hugo_Symbol=='TP53']

In [ ]:
len(inprevnotinunfiltered)

In [ ]:
inprevnotinunfiltered

In [ ]:
found = []
for k, val in prev[prev.Hugo_Symbol=='TP53'].iterrows():
    print(k,end='\r')
    for i, v in newp[newp.Hugo_Symbol=='TP53'].iterrows():
        if val.Genome_Change == v.Genome_Change:
            if val.DepMap_ID == v.DepMap_ID:
                found.append(k)
                break

In [ ]:
inprevnotinnew = prev[~prev.index.isin(found)][prev.Hugo_Symbol=='TP53']

In [ ]:
len(inprevnotinnew)

In [ ]:
inprevnotinnew.isCOSMIChotspot

In [ ]:
inprevnotinnew[inprevnotinnew.CGA_WES_AC=='0:0']

In [ ]:
prev[prev.CGA_WES_AC=='0:0']

In [ ]:
set(prev[prev.CGA_WES_AC=='0:0'].Genome_Change)

In [ ]:
new[new.Genome_Change=='g.chr17:7577548C>T']

In [ ]:
np.where(((prev[prev.Hugo_Symbol=='TP53'].Genome_Change == \
newp[newp.Hugo_Symbol=='TP53'].Genome_Change) & \
         (newp[newp.Hugo_Symbol=='TP53'].DepMap_ID == \
         newp[newp.Hugo_Symbol=='TP53'].DepMap_ID)))